# Thresholding processing regime using pyFAI

## Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import os
import matplotlib as mpl
import pandas as pd        
import math

import sys
!{sys.executable} -m pip install pyFAI
import pyFAI
from pyFAI.gui import jupyter

!{sys.executable} -m pip install pybaselines
from pybaselines import Baseline
from pybaselines.utils import gaussian

pi = math.pi

## Specify diffraction frame to read in

In [ ]:
# data file from I11 in form i11-1-61284.nxs, adjust depending on data name and format


data_file_no = "61284"

data_dir = "C:/path/to/your/data/file/"

data_file = "i11-1-" + data_file_no + ".nxs"

file_path = os.path.join(data_dir, data_file)

In [ ]:
# creation of folder to store processed information
processing_folder = "processed_patterns/individual_frames/"
processing_path = os.path.join(data_dir,processing_folder)


if not os.path.exists(processing_path):
    os.makedirs(processing_path)

In [ ]:
with h5py.File(file_path, 'r') as dat:
    
    #load in as (1, 1441, 1440) shape array
    myData = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
    
    #extract only (1441,1440)
    diff_frame = myData.reshape(myData.shape[1:])
    
total_intensity = np.sum(diff_frame)

diff_frame

In [ ]:
# visualise chosen diffraction frame

plt.rcParams["figure.figsize"]=14,14
plt.title(data_file, fontsize=24)
plt.imshow(diff_frame, cmap='gray', vmin=0, vmax=300)
plt.colorbar()
plt.show()

In [ ]:
# specify background directory location from diffraction_sorting.py step for collection set
# also need to be .nxs files

background_dir = "C:/path/to/your/background/frames/"
count = 0
bg_file_nxs = []

# Count the number of .nxs files in directory
for files in os.listdir(background_dir):
    filename = os.fsdecode(files)
    if filename.endswith(".nxs"):
        bg_file_nxs.append(filename)
        count += 1
print('File count:', count)
print(bg_file_nxs)


In [ ]:
# read in background frames and calculate total counts

bg_frame_sized = np.zeros((count, 1441, 1440))
basic_bg_sub = np.zeros((count, 1441, 1440))
total_intensity = np.zeros(count)
a = 0

while a < count:
    with h5py.File(background_dir+bg_file_nxs[a], 'r') as dat:
        background_frame = np.array(dat["/entry1/pixium_hdf/data"][()][:]) 
        bg_frame_sized[a] = background_frame.reshape(background_frame.shape[1:])
        basic_bg_sub[a] = np.subtract(diff_frame,bg_frame_sized[a])
        basic_bg_sub[a][basic_bg_sub[a] < 0] = 0
        total_intensity[a] = np.sum(basic_bg_sub[a])
        a += 1


In [ ]:
# visualisation of background best fit checks. Edit subplots (2,3) depending on plots to visualise

#fig, axs = plt.subplots(2,5, figsize=(15, 6), facecolor='w', edgecolor='k')
fig, axs = plt.subplots(2,3, figsize=(15, 20), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)

axs = axs.ravel()

for i in range(count):
    fig.suptitle('Basic background subraction best fit check', fontsize=24)
    axs[i].imshow(basic_bg_sub[i],cmap='gray',vmin=0, vmax=100)
    axs[i].set_title(bg_file_nxs[i]+"\nTotal intensity:"+str(round(total_intensity[i])), fontweight= 'bold')

In [ ]:
# autoselection of best-fit background

chosen_bg_name = bg_file_nxs[np.argmin(total_intensity)]

chosen_bg = bg_frame_sized[np.argmin(total_intensity)]

print('Chosen background: ' + str(chosen_bg_name) + '\nTotal intensity: ' + str(total_intensity[np.argmin(total_intensity)]))
#print('chosen background:' + chosen_bg)

In [ ]:
# visualisation of chosen background frame

plt.rcParams["figure.figsize"]=14,14
plt.title(chosen_bg_name, fontsize=24)
plt.imshow(chosen_bg, cmap='gray', vmin=0, vmax=500)
plt.colorbar()
plt.show()

In [ ]:
# background subtraction of background frame from diffraction frame

pattern_process = diff_frame - chosen_bg

basic_pattern_name = data_file_no + "_bg_" + chosen_bg_name[6:-4]
basic_pattern_name

In [ ]:
# visualisation of basic background subtraction

plt.rcParams["figure.figsize"]=14,14
plt.title(basic_pattern_name, fontsize=24)
plt.imshow(pattern_process, cmap='gray', vmin=0, vmax=100)
plt.colorbar()
#plt.savefig(processing_path + basic_pattern_name +".png")
plt.show()

# Thresholding

In [ ]:
# thresholding to remove remaining noise - change threshold_value

pattern_processing = diff_frame - chosen_bg

threshold_value = 120

pattern_processing[pattern_processing < threshold_value] = 0

thresholded_frame = pattern_processing

In [ ]:
# visualise thresholding, change threshold value if noise remains

threshold_pattern_name = data_file_no + "_bg_" + chosen_bg_name[6:-4] + "_threshold_" + str(threshold_value)

plt.rcParams["figure.figsize"]=14,14
plt.title(threshold_pattern_name, fontsize=24)
plt.imshow(thresholded_frame, cmap='gray', vmin=20, vmax=100)
plt.colorbar()
#plt.savefig(processing_path + threshold_pattern_name + ".png")
plt.show()

# Load calibration file

calibrated using pyFAI-calib2 tool

pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
# load the calibration .poni file 
calibration = pyFAI.load('C:/poni/file/location/calibration.poni')
calibration

In [ ]:
#load mask for detector frame
oneD_mask = np.load("C:/mask/file/location/mask.npy")

## Integration using pyFAI module
#### pyFAI: https://pyfai.readthedocs.io/en/v2023.1/

In [ ]:
#integrate pattern using pyFAI and display. Re-threshold if noise remains

oneD_integrated_plot_thresholded = calibration.integrate1d(thresholded_frame, 1000, unit=pyFAI.units.TTH_DEG, radial_range=[0,30], mask=mask.npy)
jupyter.plot1d(oneD_integrated_plot_thresholded)
plt.title(threshold_pattern_name, fontsize=15)
plt.xlabel('$2\\theta$ ($^{o}$)')
plt.xlim(4, 30)
plt.ylabel('Intensity')
#plt.savefig(processing_path + threshold_pattern_name +"_1D_plot")

In [ ]:
# export processed frame as .xy file

np.savetxt(processing_path+threshold_pattern_name + ".xy", np.c_[oneD_integrated_plot_thresholded.radial, oneD_integrated_plot_thresholded.intensity])